# Auditor de Organismos del IPD

Notebook interactivo para analizar estrategias evolucionadas de la simulación del Dilema del Prisionero Iterado.
Carga organismos una sola vez y permite exploración, prueba y comparación eficientes.

## 1. Configuración - Inicializar Entorno

In [ ]:
import os
import sys
import glob
import numpy as np
from pathlib import Path
from datetime import datetime

# Agregar raíz del proyecto a la ruta
RAIZ_PROYECTO = '/Users/pauloc/Masters/1_Sem/IA/Proyecto'
sys.path.insert(0, RAIZ_PROYECTO)

from organism_serializer import OrganismReconstructior
print("Módulos importados correctamente")

## 2. Cargar Experimento y Almacenar en Caché Organismos

Establece la ruta del experimento y carga todos los organismos en memoria (se hace una sola vez, reutilizable en la sesión)

In [ ]:
# Configuración: Cambiar esto a la ruta de tu experimento
RUTA_EXPERIMENTOS = "/Users/pauloc/Masters/1_Sem/IA/Proyecto/experiments/"  # Directorio padre
NOMBRE_EXPERIMENTO = "ipd_duel_20251121_221548"  # Dejar None para auto-detectar el más reciente, o establecer el nombre específico de la carpeta del experimento

# Auto-detectar el experimento más reciente si no se especifica
if NOMBRE_EXPERIMENTO is None:
    experimentos = sorted([d for d in os.listdir(RUTA_EXPERIMENTOS) if os.path.isdir(os.path.join(RUTA_EXPERIMENTOS, d))])
    if experimentos:
        NOMBRE_EXPERIMENTO = experimentos[-1]  # Obtener el más reciente
        print(f"Experimento auto-detectado: {NOMBRE_EXPERIMENTO}")
    else:
        print(f"ERROR: No se encontraron experimentos en {RUTA_EXPERIMENTOS}")
        sys.exit(1)

ruta_experimento_completa = os.path.join(RUTA_EXPERIMENTOS, NOMBRE_EXPERIMENTO)
carpeta_organismos = os.path.join(ruta_experimento_completa, "organisms")

if not os.path.exists(carpeta_organismos):
    print(f"ERROR: Carpeta de organismos no encontrada: {carpeta_organismos}")
    sys.exit(1)

print(f"\nExperimento: {NOMBRE_EXPERIMENTO}")
print(f"Ruta: {ruta_experimento_completa}")

In [ ]:
# Cargar todas las rutas de archivos de organismos y almacenar en caché
archivos_organismos = sorted(glob.glob(os.path.join(carpeta_organismos, "*.pkl")))
cache_organismos = {}  # Caché en memoria para organismos cargados
cache_info_organismos = {}  # Caché de información rápida (sin reconstrucción completa)

print(f"\nSe encontraron {len(archivos_organismos)} organismos")
print("\nÍndice de Organismos:")
print("-" * 110)
print(f"{'Índice':<8} {'Archivo':<60} {'Memoria':<10} {'Parámetros':<12}")
print("-" * 110)

for idx, archivo_pkl in enumerate(archivos_organismos):
    nombre_archivo = os.path.basename(archivo_pkl)
    try:
        info = OrganismReconstructior.get_organism_info(archivo_pkl)
        cache_info_organismos[idx] = info
        cache_info_organismos[idx]['ruta_archivo'] = archivo_pkl
        memoria = info['tipo_jugador']
        parametros = info['num_weights'] + info['num_biases']
        print(f"{idx:<8} {nombre_archivo:<60} {memoria:<10} {parametros:<12}")
    except Exception as e:
        print(f"{idx:<8} {nombre_archivo:<60} ERROR: {str(e)}")

print("-" * 110)
print(f"\nÍndice de organismos cargado (usar índices 0-{len(archivos_organismos)-1} para referenciar organismos)")

## 3. Funciones Auxiliares

Utilidades refactorizadas desde la CLI

In [ ]:
def cargar_organismo(idx):
    """Cargar organismo por índice (almacenado en caché para eficiencia)"""
    if idx not in cache_organismos:
        if idx not in cache_info_organismos:
            print(f"ERROR: Índice de organismo inválido {idx}")
            return None
        ruta_archivo = cache_info_organismos[idx]['ruta_archivo']
        cache_organismos[idx] = OrganismReconstructior.reconstruct_from_pickle(ruta_archivo)
    return cache_organismos[idx]

def inspeccionar_organismo(idx):
    """Mostrar información detallada sobre un organismo"""
    if idx not in cache_info_organismos:
        print(f"ERROR: Índice de organismo inválido {idx}")
        return
    
    info = cache_info_organismos[idx]
    organismo = cargar_organismo(idx)
    
    print(f"\n{'='*80}")
    print(f"ORGANISMO {idx} - {os.path.basename(info['ruta_archivo'])}")
    print(f"{'='*80}")
    print(f"Tipo de memoria:       {info['tipo_jugador']}")
    print(f"Arquitectura:          {info['capas_ocultas']}")
    print(f"Generación:            {info['generacion']}")
    print(f"ID del jugador:        {info['jugador_id']}")
    print(f"Población:             {info['tipo_poblacion'] if info['tipo_poblacion'] else 'N/A'}")
    print(f"Total de pesos:        {info['num_weights']}")
    print(f"Total de sesgos:       {info['num_biases']}")
    print(f"Total de parámetros:   {info['num_weights'] + info['num_biases']}")
    
    print(f"\n{'Resumen de Pesos de la Red Neuronal:'}")
    print("-" * 80)
    for idx_capa, (pesos, sesgos) in enumerate(
        zip(organismo.decision.pesos, organismo.decision.sesgos)
    ):
        print(f"Capa {idx_capa}:")
        print(f"  Forma de pesos:  {pesos.shape}")
        print(f"  Rango de pesos:  [{pesos.min():.4f}, {pesos.max():.4f}]")
        print(f"  Media de pesos:  {pesos.mean():.4f}")
        print(f"  Forma de sesgos: {sesgos.shape}")
        print(f"  Rango de sesgos: [{sesgos.min():.4f}, {sesgos.max():.4f}]")
        print()

## 4. Probar Organismo - Análisis de Decisiones

Prueba cómo un organismo responde a varios historiales de juego

In [ ]:
def probar_organismo(idx, historiales_prueba=None):
    """Probar la toma de decisiones del organismo con historiales de muestra"""
    if idx not in cache_info_organismos:
        print(f"ERROR: Índice de organismo inválido {idx}")
        return
    
    organismo = cargar_organismo(idx)
    info = cache_info_organismos[idx]
    
    # Historiales de prueba por defecto si no se proporcionan
    if historiales_prueba is None:
        historiales_prueba = [
            [[], []],  # Historial vacío (inicio del juego)
            [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]],  # Ambos cooperaron
            [[1, 1, 1, 1, 1], [1, 1, 1, 1, -1]],  # Cooperación mixta
            [[1, -1, -1, -1, -1], [1, 1, 1, 1, 1]],  # Explotado
            [[1, 1, 1, 1, 1], [-1, -1, -1, -1, -1]],  # Oponente siempre deserta
            [[1, 1, -1, 1, -1], [1, -1, 1, -1, 1]],  # Aleatorio/alternando
        ]
    
    print(f"\n{'='*80}")
    print(f"PRUEBA DE DECISIÓN - Organismo {idx}")
    print(f"Tipo de memoria: {info['tipo_jugador']} | Arquitectura: {info['capas_ocultas']}")
    print(f"{'='*80}")
    print(f"{'Prueba':<8} {'Historial propio':<25} {'Historial oponente':<25} {'Decisión':<15}")
    print("-" * 80)
    
    for i, historial in enumerate(historiales_prueba):
        decision = organismo.jugar(historial)
        accion = "COOPERA (1)" if decision == 1 else "DESERTA (-1)"
        hist_propio = str(historial[0][-5:]) if historial[0] else "[]"  # Mostrar últimos 5
        hist_oponente = str(historial[1][-5:]) if historial[1] else "[]"  # Mostrar últimos 5
        print(f"{i+1:<8} {hist_propio:<25} {hist_oponente:<25} {accion:<15}")

print("Función probar_organismo() lista")

## 5. Comparar Organismos

Comparación lado a lado de dos organismos

In [ ]:
def comparar_organismos(idx1, idx2, historiales_prueba=None):
    """Comparar dos organismos lado a lado"""
    if idx1 not in cache_info_organismos or idx2 not in cache_info_organismos:
        print(f"ERROR: Índices de organismo inválidos")
        return
    
    info1 = cache_info_organismos[idx1]
    info2 = cache_info_organismos[idx2]
    org1 = cargar_organismo(idx1)
    org2 = cargar_organismo(idx2)
    
    # Historiales de prueba por defecto
    if historiales_prueba is None:
        historiales_prueba = [
            [[], []],
            [[1, 1], [1, 1]],
            [[1, -1], [-1, 1]],
            [[-1, -1, -1], [-1, -1, -1]],
        ]
    
    print(f"\n{'='*100}")
    print(f"COMPARACIÓN DE ORGANISMOS")
    print(f"{'='*100}")
    
    print(f"\nOrganismo {idx1}: {os.path.basename(info1['ruta_archivo'])}")
    print(f"  Memoria: {info1['tipo_jugador']}, Parámetros: {info1['num_weights'] + info1['num_biases']}")
    print(f"\nOrganismo {idx2}: {os.path.basename(info2['ruta_archivo'])}")
    print(f"  Memoria: {info2['tipo_jugador']}, Parámetros: {info2['num_weights'] + info2['num_biases']}")
    
    print(f"\n{'Decisiones de muestra en la misma entrada:'}")
    print("-" * 100)
    print(f"{'Prueba':<8} {'Historial propio':<20} {'Historial oponente':<20} {'Org1':<15} {'Org2':<15} {'Coincidencia':<15}")
    print("-" * 100)
    
    coincidencias = 0
    for i, hist in enumerate(historiales_prueba):
        dec1 = org1.jugar(hist)
        dec2 = org2.jugar(hist)
        act1 = "COOP" if dec1 == 1 else "DEF"
        act2 = "COOP" if dec2 == 1 else "DEF"
        coincidencia = "COINCIDEN" if dec1 == dec2 else "DIFIEREN"
        if dec1 == dec2:
            coincidencias += 1
        propio = str(hist[0][-3:]) if hist[0] else "[]"
        opon = str(hist[1][-3:]) if hist[1] else "[]"
        print(f"{i+1:<8} {propio:<20} {opon:<20} {act1:<15} {act2:<15} {coincidencia:<15}")
    
    print("-" * 100)
    print(f"Decisiones coincidentes: {coincidencias}/{len(historiales_prueba)}")

print("Función comparar_organismos() lista")

## 6. Jugar Partida - Simulación de Juego Interactivo

Simula una partida completa del IPD entre dos organismos con visualización detallada movimiento a movimiento

In [ ]:
def jugar_partida(idx1, idx2, rondas=100, mostrar_todo=False):
    """
    Simula una partida entre dos organismos.
    
    Parámetros:
    - idx1, idx2: Índices de organismos
    - rondas: Número de rondas a jugar
    - mostrar_todo: Si es False, muestra estadísticas resumidas + cada 10ª ronda. Si es True, muestra todas las rondas.
    """
    if idx1 not in cache_info_organismos or idx2 not in cache_info_organismos:
        print(f"ERROR: Índices de organismo inválidos")
        return
    
    info1 = cache_info_organismos[idx1]
    info2 = cache_info_organismos[idx2]
    org1 = cargar_organismo(idx1)
    org2 = cargar_organismo(idx2)
    
    print(f"\n{'='*100}")
    print(f"PARTIDA - Organismo {idx1} vs Organismo {idx2}")
    print(f"{'='*100}")
    print(f"\nJugador 1: {os.path.basename(info1['ruta_archivo'])}")
    print(f"  Memoria: {info1['tipo_jugador']}, Arquitectura: {info1['capas_ocultas']}")
    print(f"  Total de parámetros: {info1['num_weights'] + info1['num_biases']}")
    print(f"\nJugador 2: {os.path.basename(info2['ruta_archivo'])}")
    print(f"  Memoria: {info2['tipo_jugador']}, Arquitectura: {info2['capas_ocultas']}")
    print(f"  Total de parámetros: {info2['num_weights'] + info2['num_biases']}")
    print(f"\nPartida: {rondas} rondas | Puntuación: CC=3,3 | CD=5,0 | DC=0,5 | DD=1,1")
    
    # Jugar la partida
    historial1 = [[], []]
    historial2 = [[], []]
    puntos1 = 0
    puntos2 = 0
    movimientos = []
    
    for num_ronda in range(rondas):
        # Obtener decisiones
        dec1 = org1.jugar(historial1)
        dec2 = org2.jugar(historial2)
        
        # Calcular puntos
        if dec1 == 1 and dec2 == 1:
            p1, p2 = 3, 3
        elif dec1 == -1 and dec2 == 1:
            p1, p2 = 5, 0
        elif dec1 == 1 and dec2 == -1:
            p1, p2 = 0, 5
        else:
            p1, p2 = 1, 1
        
        puntos1 += p1
        puntos2 += p2
        
        # Actualizar historiales
        historial1[0].append(dec1)
        historial1[1].append(dec2)
        historial2[0].append(dec2)
        historial2[1].append(dec1)
        
        # Guardar movimiento para mostrar
        movimientos.append((dec1, dec2, p1, p2))
    
    # Mostrar reproducción de la partida
    print(f"\n{'='*100}")
    print(f"REPRODUCCIÓN DE LA PARTIDA (C = Coopera, D = Deserta)")
    print(f"{'='*100}")
    print(f"{'Ronda':<8} {'J1':<5} {'J2':<5} {'Puntos J1':<12} {'Puntos J2':<12} {'Total J1':<12} {'Total J2':<12}")
    print("-" * 100)
    
    acumulado_p1, acumulado_p2 = 0, 0
    for i, (mov1, mov2, s1, s2) in enumerate(movimientos, 1):
        acumulado_p1 += s1
        acumulado_p2 += s2
        
        # Mostrar cada ronda si mostrar_todo, sino cada 10ª + primera y última
        debe_mostrar = mostrar_todo or i == 1 or i == rondas or i % 10 == 0
        
        if debe_mostrar:
            accion1 = "C" if mov1 == 1 else "D"
            accion2 = "C" if mov2 == 1 else "D"
            print(f"{i:<8} {accion1:<5} {accion2:<5} {s1:<12} {s2:<12} {acumulado_p1:<12} {acumulado_p2:<12}")
        elif i > 1 and (i-1) % 10 == 0:  # Mostrar indicador de salto
            print(f"...")
    
    # Mostrar puntuaciones finales
    print("=" * 100)
    print(f"{'PUNTUACIÓN FINAL':<8} {' ':<5} {' ':<5} {' ':<12} {' ':<12} {puntos1:<12} {puntos2:<12}")
    print("=" * 100)
    
    # Determinar ganador
    if puntos1 > puntos2:
        print(f"\nGANADOR: Jugador 1 (Org {idx1}) por {puntos1 - puntos2} puntos")
    elif puntos2 > puntos1:
        print(f"\nGANADOR: Jugador 2 (Org {idx2}) por {puntos2 - puntos1} puntos")
    else:
        print(f"\nEMPATE: Ambos jugadores anotaron {puntos1} puntos")
    
    # Estadísticas adicionales
    print(f"\nEstadísticas:")
    p1_cooperaciones = sum(1 for m1, m2, _, _ in movimientos if m1 == 1)
    p2_cooperaciones = sum(1 for m1, m2, _, _ in movimientos if m2 == 1)
    cooperacion_mutua = sum(1 for m1, m2, _, _ in movimientos if m1 == 1 and m2 == 1)
    desercion_mutua = sum(1 for m1, m2, _, _ in movimientos if m1 == -1 and m2 == -1)
    
    print(f"  Tasa de cooperación J1: {p1_cooperaciones}/{rondas} ({100*p1_cooperaciones/rondas:.1f}%)")
    print(f"  Tasa de cooperación J2: {p2_cooperaciones}/{rondas} ({100*p2_cooperaciones/rondas:.1f}%)")
    print(f"  Cooperación mutua:      {cooperacion_mutua}/{rondas} ({100*cooperacion_mutua/rondas:.1f}%)")
    print(f"  Deserción mutua:        {desercion_mutua}/{rondas} ({100*desercion_mutua/rondas:.1f}%)")
    print(f"  Puntuación promedio/ronda: J1={puntos1/rondas:.2f}, J2={puntos2/rondas:.2f}")

print("Función jugar_partida() lista")

---

# Ejemplos de Uso Interactivo

Usa las funciones a continuación para analizar tus organismos. Modifica índices y parámetros según sea necesario.

### Ejemplo: Inspeccionar un organismo

In [ ]:
# Reemplazar con el índice de organismo deseado
inspeccionar_organismo(49359)

### Ejemplo: Probar decisiones del organismo

In [ ]:
# Probar organismo 0 con casos de prueba por defecto
probar_organismo(49350)

# O definir historiales de prueba personalizados:
# historiales_personalizados = [
#     [[], []],  # Vacío
#     [[1, 1, 1], [1, 1, 1]],  # Ambos cooperativos
# ]
# probar_organismo(0, historiales_personalizados)

### Ejemplo: Comparar dos organismos

In [ ]:
# Comparar organismo 0 vs organismo 1
comparar_organismos(49350, 49351)

### Ejemplo: Jugar una partida

#### Tercera etapa: Fase de traición

In [ ]:
# Jugar 100 rondas entre organismo 0 y organismo 1
jugar_partida(49302, 49355, rondas=10, mostrar_todo=True)

# Usa mostrar_todo=True para ver cada ronda individual (verboso!)
# jugar_partida(0, 1, rondas=50, mostrar_todo=True)

#### Etapa 2: segundo pico de cooperación

In [ ]:
# Jugar 100 rondas entre organismo 0 y organismo 1
jugar_partida(45012, 45063, rondas=10, mostrar_todo=True)

# Usa mostrar_todo=True para ver cada ronda individual (verboso!)
# jugar_partida(0, 1, rondas=50, mostrar_todo=True)

#### Etapa 1: primer pico de cooperación

In [ ]:
# Jugar 100 rondas entre organismo 0 y organismo 1
jugar_partida(30014, 30055, rondas=10, mostrar_todo=True)

# Usa mostrar_todo=True para ver cada ronda individual (verboso!)
# jugar_partida(0, 1, rondas=50, mostrar_todo=True)

### Ejemplo: Probar diferentes organismos entre sí

In [ ]:
# Estilo torneo: probar los mejores organismos entre sí
indices_mejores = [0, 1, 2, 3, 4]  # Modificar basado en tus organismos mejores

print("\n" + "="*100)
print("TORNEO: Probando Organismos Mejores")
print("="*100)

resultados = {}  # Almacenar resultados de partidas

for i, idx1 in enumerate(indices_mejores):
    resultados[idx1] = {'victorias': 0, 'derrotas': 0, 'empates': 0, 'puntos': 0}
    
    for idx2 in indices_mejores:
        if idx1 >= idx2:  # Solo probar cada par una vez
            continue
        
        # Partida rápida (50 rondas para ahorrar tiempo)
        historial1 = [[], []]
        historial2 = [[], []]
        puntos1, puntos2 = 0, 0
        
        org1 = cargar_organismo(idx1)
        org2 = cargar_organismo(idx2)
        
        for _ in range(50):
            dec1 = org1.jugar(historial1)
            dec2 = org2.jugar(historial2)
            
            if dec1 == 1 and dec2 == 1:
                p1, p2 = 3, 3
            elif dec1 == -1 and dec2 == 1:
                p1, p2 = 5, 0
            elif dec1 == 1 and dec2 == -1:
                p1, p2 = 0, 5
            else:
                p1, p2 = 1, 1
            
            puntos1 += p1
            puntos2 += p2
            
            historial1[0].append(dec1)
            historial1[1].append(dec2)
            historial2[0].append(dec2)
            historial2[1].append(dec1)
        
        resultados[idx1]['puntos'] += puntos1
        if puntos1 > puntos2:
            resultados[idx1]['victorias'] += 1
        elif puntos1 < puntos2:
            resultados[idx1]['derrotas'] += 1
        else:
            resultados[idx1]['empates'] += 1

# Mostrar resultados del torneo
print(f"\n{'Org':<6} {'Victorias':<8} {'Derrotas':<8} {'Empates':<8} {'Puntos Totales':<15}")
print("-" * 50)
for org_idx in sorted(resultados.keys(), key=lambda x: resultados[x]['puntos'], reverse=True):
    r = resultados[org_idx]
    print(f"{org_idx:<6} {r['victorias']:<8} {r['derrotas']:<8} {r['empates']:<8} {r['puntos']:<15}")